In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re
# Enable inline plots
%matplotlib inline

In [ ]:
def preprocess(df):
    # Text Standardization
    # Convert `question` and `evidence` fields to lowercase
    df['question'] = df['question'].str.lower()
    df['evidence'] = df['evidence'].str.lower()
    # Check standardized text
    df[['question', 'evidence']].head()
    # Data Cleaning
    # Handle missing values in `evidence`
    df['evidence'] = df['evidence'].fillna('missing')
    # Remove duplicate questions
    df.drop_duplicates(subset=['question'], inplace=True)
    # Feature Engineering
    # Compute SQL query length
    df['sql_length'] = df['SQL'].apply(lambda x: len(x) if pd.notnull(x) else 0)
    # Encode `difficulty` into numeric values
    difficulty_mapping = {"simple": 1, "moderate": 2, "challenging": 3}
    df['difficulty_encoded'] = df['difficulty'].map(difficulty_mapping)
    # Compute question word length
    df['question_length'] = df['question'].apply(lambda x: len(x.split()))
    # Display new features
    df[['sql_length', 'difficulty_encoded', 'question_length']].head()
    # Normalize Length Features
    # Normalize `question_length` and `sql_length`
    df['question_length_norm'] = (df['question_length'] - df['question_length'].mean()) / df['question_length'].std()
    df['sql_length_norm'] = (df['sql_length'] - df['sql_length'].mean()) / df['sql_length'].std()

    # Display normalized features
    df[['question_length_norm', 'sql_length_norm']].head()
    return df



def extract_ctes(query):
    # Matches patterns like: WITH alias AS ( ... )
    # Using a non-greedy approach to capture each CTE block.
    # This regex will find all occurrences of "WITH ... AS ( ... )" and subsequent CTEs 
    # if chained with commas.
    cte_pattern = r'WITH\s+(.*?)\s+AS\s*\((.*?)\)(?:,|$)'
    # Using DOTALL to span multiple lines
    matches = re.findall(cte_pattern, query, flags=re.IGNORECASE|re.DOTALL)
    
    ctes = {}
    for alias, cte_subquery in matches:
        alias = alias.strip()
        ctes[alias.lower()] = cte_subquery
    return ctes

def extract_table_references(sql):
    # This captures any word after FROM or JOIN. Adjust pattern if needed to match your table naming conventions.
    return re.findall(r'(?:FROM|JOIN)\s+([A-Za-z_][A-Za-z0-9_]*)', sql, flags=re.IGNORECASE)

def extract_all_tables(query, known_tables):
    # Extract CTEs first
    ctes = extract_ctes(query)
    
    cte_aliases = set(ctes.keys())
    all_tables = set()
    
    # Extract tables from each CTE definition
    for cte_alias, cte_sql in ctes.items():
        cte_tables = extract_table_references(cte_sql)
        for tbl in cte_tables:
            all_tables.add(tbl.lower())

    # Now extract from the main query (after all CTEs)
    # Strip the WITH clauses (optional, depends on how you parse)
    # A crude approach: remove the WITH clauses and what follows until main SELECT
    # If the query structure is stable, you can isolate the main query part.
    main_query_part = re.split(r'WITH\s+', query, flags=re.IGNORECASE|re.DOTALL)
    if len(main_query_part) > 1:
        # After splitting by WITH, the main query typically is after the last ) 
        # that closes the last CTE. We can try to find that:
        main_query = re.split(r'\)\s*SELECT', query, flags=re.IGNORECASE|re.DOTALL)
        if len(main_query) > 1:
            main_query = 'SELECT' + main_query[-1]  # re-attach SELECT
        else:
            # fallback if regex above doesn't isolate well
            main_query = query
    else:
        # no WITH clause
        main_query = query
    
    main_tables = extract_table_references(main_query)
    for tbl in main_tables:
        all_tables.add(tbl.lower())
    
    # Now we have all table-like references. Some are actual tables, some are CTE aliases.
    # Filter them:
    real_table_refs = [t for t in all_tables if t in known_tables]
    cte_table_refs = [t for t in all_tables if t in cte_aliases]
    invalid_refs = [t for t in all_tables if (t not in known_tables and t not in cte_aliases)]
    
    return real_table_refs, cte_table_refs, invalid_refs

def validate_table_names(tables, table_names, cte_aliases=set()):
    known_tables = set(t.lower() for t in tables)
    return all((t in known_tables or t in cte_aliases) for t in map(str.lower, table_names))

# def validate_table_names(tables, table_names):
#     # Check if all table names are valid
#     return all(table in tables for table in list(map(str.lower, table_names)))

## Load Data

In [23]:
# Load Dataset
with open("dev.json", "r") as file:
    data = json.load(file)

# Load tables
with open("dev_tables.json", "r") as file:
    tables = json.load(file)

# Extract table names
tablenames = set()
for db in tables:
    for table in db['table_names_original']:
        tablenames.add(table.lower())

# Convert JSON to DataFrame
df = pd.DataFrame(data)


In [24]:
df = preprocess(df)
df.head()

,question_id,db_id,question,evidence,SQL,difficulty,sql_length,difficulty_encoded,question_length,question_length_norm,sql_length_norm
0,0,california_schools,what is the highest eligible free rate for k-1...,eligible free rate for k-12 = `free meal count...,SELECT `Free Meal Count (K-12)` / `Enrollment ...,simple,180,1,16,0.258897,0.210309
1,1,california_schools,please list the lowest three eligible free rat...,eligible free rates for students aged 5-17 = `...,SELECT `Free Meal Count (Ages 5-17)` / `Enroll...,moderate,280,2,15,0.079876,1.348060
2,2,california_schools,please list the zip code of all the charter sc...,charter schools refers to `charter school (y/n...,SELECT T2.Zip FROM frpm AS T1 INNER JOIN schoo...,simple,178,1,16,0.258897,0.187554
3,3,california_schools,what is the unabbreviated mailing street addre...,,SELECT T2.MailStreet FROM frpm AS T1 INNER JOI...,simple,133,1,18,0.616937,-0.324433
4,4,california_schools,please list the phone numbers of the direct ch...,charter schools refers to `charter school (y/n...,SELECT T2.Phone FROM frpm AS T1 INNER JOIN sch...,moderate,200,2,15,0.079876,0.437860


In [25]:
for idx, row in df[["SQL", "db_id"]].iterrows():
    # Extract tables from SQL
    extracted_table_names = extract_all_tables(row.SQL, tablenames)
    table_names, cte_aliases, invalid_refs = extracted_table_names
    if not validate_table_names(tablenames, table_names, cte_aliases):
        print(f"Invalid table name in row {idx}: {table_names}")
        print("DB: ",row.db_id)
        print(f"\t\t{row.SQL}")
    # if invalid_refs:
    #     print(f"Invalid table reference in row {idx}: {invalid_refs}")
    #     print("DB: ",row.db_id)
    #     print(f"\t\t{row.SQL}")
    # if cte_aliases:
    #     print(f"CTE aliases in row {idx}: {cte_aliases}")
    #     print("DB: ",row.db_id)
    #     print(f"\t\t{row.SQL}")

In [ ]:
df_table_names = df.copy()
df_table_names = df.drop(columns=["difficulty","sql_length", "difficulty_encoded", "question_length", "question_length_norm", "sql_length_norm"])
df_table_names["table_names"] = df_table_names["SQL"].apply(lambda x: extract_all_tables(x, tablenames)[0])
df_table_names["cte_aliases"] = df_table_names["SQL"].apply(lambda x: extract_all_tables(x, tablenames)[1])

In [38]:
df_table_names.head()

,question_id,db_id,question,evidence,SQL,table_names,cte_aliases
0,0,california_schools,what is the highest eligible free rate for k-1...,eligible free rate for k-12 = `free meal count...,SELECT `Free Meal Count (K-12)` / `Enrollment ...,[frpm],[]
1,1,california_schools,please list the lowest three eligible free rat...,eligible free rates for students aged 5-17 = `...,SELECT `Free Meal Count (Ages 5-17)` / `Enroll...,[frpm],[]
2,2,california_schools,please list the zip code of all the charter sc...,charter schools refers to `charter school (y/n...,SELECT T2.Zip FROM frpm AS T1 INNER JOIN schoo...,"[schools, frpm]",[]
3,3,california_schools,what is the unabbreviated mailing street addre...,,SELECT T2.MailStreet FROM frpm AS T1 INNER JOI...,"[schools, frpm]",[]
4,4,california_schools,please list the phone numbers of the direct ch...,charter schools refers to `charter school (y/n...,SELECT T2.Phone FROM frpm AS T1 INNER JOIN sch...,"[schools, frpm]",[]


In [40]:
# Save Processed Dataset
# Save the processed dataset to a CSV file
df_table_names.to_csv("tablenames_dataset.csv", index=False)
# Confirm save
print("Processed dataset saved as 'processed_dataset.csv'.")

Processed dataset saved as 'processed_dataset.csv'.


In [80]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from scipy.sparse import hstack  # For combining sparse matrices

# Load the dataset
df = pd.read_csv('tablenames_dataset.csv')

# Preprocess the dataset
df['input_text'] = df['db_id'] + " " + df['question'] + " " + df['evidence']
df['primary_table'] = df['table_names'].apply(lambda x: eval(x)[0] if eval(x) else None)

# Drop rows with missing input_text or primary_table values
df_cleaned = df.dropna(subset=['input_text', 'primary_table'])

# Prepare features and target
X_text = df_cleaned['input_text']
X_db = pd.get_dummies(df_cleaned['db_id'], sparse=True)  # One-hot encode db_id
y = df_cleaned['primary_table']

# Train-test split
X_text_train, X_text_test, X_db_train, X_db_test, y_train, y_test = train_test_split(
    X_text, X_db, y, test_size=0.2, random_state=42
)

# Vectorize text input
vectorizer = TfidfVectorizer(max_features=5000)
X_text_train_vec = vectorizer.fit_transform(X_text_train)
X_text_test_vec = vectorizer.transform(X_text_test)

# Combine TF-IDF features with db_id encoding
X_train_combined = hstack([X_text_train_vec, X_db_train])
X_test_combined = hstack([X_text_test_vec, X_db_test])

# Define baseline models
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "NaiveBayes": MultinomialNB()
}

# Evaluate each model
baseline_results = {}
for name, model in models.items():
    # Train the model
    model.fit(X_train_combined, y_train)
    # Predict on test set
    y_pred = model.predict(X_test_combined)
    # Generate classification report
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    baseline_results[name] = {
        "Overall Accuracy": accuracy_score(y_test, y_pred),
        "Precision (Macro Avg)": report["macro avg"]["precision"],
        "Recall (Macro Avg)": report["macro avg"]["recall"],
        "F1 Score (Macro Avg)": report["macro avg"]["f1-score"]
    }

# Display results
results_df = pd.DataFrame(baseline_results).T
print(results_df.to_latex())

\begin{tabular}{lrrrr}
\toprule
 & Overall Accuracy & Precision (Macro Avg) & Recall (Macro Avg) & F1 Score (Macro Avg) \\
\midrule
RandomForest & 0.761733 & 0.497755 & 0.527802 & 0.488209 \\
LogisticRegression & 0.722022 & 0.443354 & 0.452007 & 0.411678 \\
NaiveBayes & 0.628159 & 0.206386 & 0.290850 & 0.236403 \\
\bottomrule
\end{tabular}

